In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
data = keras.datasets.imdb

In [3]:
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words = 88000)

D:\Data Science\Anaconda\envs\akram\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
D:\Data Science\Anaconda\envs\akram\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


# Making the data for human readability

In [5]:
word_index = data.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

# Preprocessing the data

In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

train_data = keras.preprocessing.sequence.pad_sequences(train_data, value = word_index["<PAD>"], padding = "post", maxlen = 250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value = word_index["<PAD>"], padding = "post", maxlen = 250)

def decode_review(text):
    return " ".join(reverse_word_index.get(i, "?") for i in text)

In [7]:
print(decode_review(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sho

In [8]:
print(len(train_data[0]), len(test_data[0]))

250 250


# Defining the model

In [9]:
model = keras.Sequential()
model.add(keras.layers.Embedding(88000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation = "relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid"))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
x_val = train_data[ : 10000]
x_train = train_data[10000 : ]

y_val = train_labels[ : 10000]
y_train = train_labels[10000 : ]

# Training the model

In [13]:
fitModel = model.fit(x_train, y_train, epochs=40, batch_size=512, validation_data=(x_val, y_val), verbose=1)

Epoch 1/40
30/30 [==============================] - 1s 48ms/step - loss: 0.6925 - accuracy: 0.5569 - val_loss: 0.6912 - val_accuracy: 0.7095
Epoch 2/40
30/30 [==============================] - 1s 41ms/step - loss: 0.6887 - accuracy: 0.7377 - val_loss: 0.6854 - val_accuracy: 0.7339
Epoch 3/40
30/30 [==============================] - 1s 39ms/step - loss: 0.6789 - accuracy: 0.7336 - val_loss: 0.6722 - val_accuracy: 0.7432
Epoch 4/40
30/30 [==============================] - 1s 39ms/step - loss: 0.6585 - accuracy: 0.7635 - val_loss: 0.6480 - val_accuracy: 0.7766
Epoch 5/40
30/30 [==============================] - 1s 39ms/step - loss: 0.6253 - accuracy: 0.7949 - val_loss: 0.6133 - val_accuracy: 0.7928
Epoch 6/40
30/30 [==============================] - 1s 39ms/step - loss: 0.5810 - accuracy: 0.8203 - val_loss: 0.5717 - val_accuracy: 0.8088
Epoch 7/40
30/30 [==============================] - 1s 39ms/step - loss: 0.5298 - accuracy: 0.8436 - val_loss: 0.5260 - val_accuracy: 0.8196
Epoch 8/40
30

In [14]:
results = model.evaluate(test_data, test_labels)

782/782 [==============================] - 2s 3ms/step - loss: 0.3381 - accuracy: 0.8723


In [15]:
print(results)

[0.33805030584335327, 0.8723199963569641]


In [ ]:
#test_review = test_data[0]
#predict = model.predict([test_review])
#print("Review: ")
#print(decode_review(test_review))
#print("Prediction: " + str(predict[0]))
#print("Actual: " + str(test_labels[0]))

In [16]:
model.save("movie_review.h5")

In [17]:
new_model = keras.models.load_model("movie_review.h5")

# Testing with outside data

In [19]:
def review_encode(string):
    encoded = [1]
    for word in string:
        if word.lower() in word_index:
            encoded.append(word_index[word.lower()])
        else:
            encoded.append(2)
    return encoded

In [26]:
with open("test.txt", encoding="utf-8") as f:
    for line in f.readlines():
        nline = line.replace(',', '').replace('.', '').replace('(', '').replace(')', '').replace(':', '').replace("\"", "").strip().split(" ")
        encode = review_encode(nline)
        encode = keras.preprocessing.sequence.pad_sequences([encode], value = word_index["<PAD>"], padding = "post",maxlen = 250)
        predict = model.predict(encode)
        print(line)
        print(encode)
        print(predict[0])

Of all the animation classics from the Walt Disney Company, there is perhaps none that is more celebrated than "The Lion King." Its acclaim is understandable: this is quite simply a glorious work of art."The Lion King" gets off to a fantastic start. The film's opening number, "The Circle of Life," is outstanding. The song lasts for about four minutes, but from the first sound, the audience is floored. Not even National Geographic can capture something this beautiful and dramatic. Not only is this easily the greatest moment in film animation, this is one of the greatest sequences in film history.The story that follows is not as majestic, but the film has to tell a story. Actually, the rest of the film holds up quite well. The story takes place in Africa, where the lions rule. Their king, Mufasa (James Earl Jones) has just been blessed with a son, Simba (Jonathan Taylor Thomas), who goes in front of his uncle Scar (Jeremy Irons) as next in line for the throne. Scar is furious, and sets i